### Binary Classifier on a single label using text_tokens as content

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import twitter_preproc

conf = SparkConf().setAll([
    ("num-executors", 4), 
    ("total-executor-cores", 16), 
    ("executor-memory", "8g"),
    ("spark.yarn.executor.memoryOverhead", "64g")])
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=yarn) created by __init__ at <ipython-input-1-5798936379fc>:10 

In [3]:
datapath = "///tmp/traintweet_10k.tsv"
user = "engaging_user_id"

Load DF and change Timestamp/None to 1/0 in target column

In [4]:
import importlib
from pyspark.sql.functions import when

importlib.reload(twitter_preproc)
preproc = twitter_preproc.twitter_preproc(spark, sc, datapath, method="CB")
df = preproc.getDF()

### Handle BERT tokens like words, maybe TODO: find deeper meaning in the tokens(embeddings)

In [5]:
from pyspark.ml.feature import RegexTokenizer,NGram,CountVectorizer,IDF,StringIndexer,Normalizer
from pyspark.ml import Pipeline

stringIndexer = StringIndexer(inputCol=user, outputCol=user+"_idx")
regexTokenizer = RegexTokenizer(inputCol="text_tokens", outputCol="terms", pattern="\t")
cv = CountVectorizer(inputCol="terms", outputCol="vector")
idf = IDF(inputCol="vector", outputCol="features")
normalizer=Normalizer(inputCol="features",outputCol="normed_features")
pipeline = Pipeline(stages=[stringIndexer,regexTokenizer, cv,idf,normalizer])

model = pipeline.fit(df)


In [6]:
data = model.transform(df)
data = data.select("normed_features","engaging_user_id_idx","tweet_id","like_timestamp")

In [ ]:
user_profiles = data.rdd.map(row->row.normed_features)

# Relevance Feedback with Rocchios method 
7, S.36

for now with a single user and selected tweeet yeet

In [7]:
tweet_id = "E7D6C5094767223F6F8789A87A1937AB"
user_id = 1.0
features = "normed_features"
target = "like_timestamp"

tweet_vector = data.where(data["tweet_id"] == tweet_id)
user_vectors = data.where(data["engaging_user_id_idx"] == user_id)

In [8]:
user_vectors.select("normed_features","like_timestamp").show(20)

+--------------------+--------------+
|     normed_features|like_timestamp|
+--------------------+--------------+
|(31642,[3,4,5,7,1...|    1581004554|
|(31642,[0,1,2,3,4...|    1581109718|
|(31642,[0,1,3,4,5...|    1581437591|
|(31642,[0,1,2,3,4...|    1581182147|
|(31642,[0,1,2,3,4...|    1581190598|
|(31642,[0,1,2,3,4...|    1580960533|
|(31642,[0,1,2,3,4...|    1581530653|
|(31642,[0,1,2,3,4...|    1581367528|
|(31642,[0,3,4,5,7...|    1581182147|
+--------------------+--------------+



## Calculate linear combination of user_vectors

In [26]:
positive_feedback = user_vectors.where(user_vectors[target].isNotNull())
negative_feedback = user_vectors.where(user_vectors[target].isNull())

In [27]:
positive_feedback.printSchema()

root
 |-- normed_features: vector (nullable = true)
 |-- engaging_user_id_idx: double (nullable = false)
 |-- tweet_id: string (nullable = true)
 |-- like_timestamp: long (nullable = true)



In [51]:
positive_feedback.select("normed_features").show(10)

+--------------------+
|     normed_features|
+--------------------+
|(31642,[3,4,5,7,1...|
|(31642,[0,1,2,3,4...|
|(31642,[0,1,3,4,5...|
|(31642,[0,1,2,3,4...|
|(31642,[0,1,2,3,4...|
|(31642,[0,1,2,3,4...|
|(31642,[0,1,2,3,4...|
|(31642,[0,1,2,3,4...|
|(31642,[0,3,4,5,7...|
+--------------------+



In [20]:
negative_feedback.select("normed_features").show(10)

+---------------+
|normed_features|
+---------------+
+---------------+



In [60]:
vectors = user_vectors.select("normed_features").rdd.map(lambda row: row.normed_features)

In [61]:
from pyspark.mllib.linalg.distributed import RowMatrix

matrix = RowMatrix(vectors)

In [64]:
stats = matrix.computeColumnSummaryStatistics()

Py4JJavaError: An error occurred while calling o275726.computeColumnSummaryStatistics.
: org.apache.spark.SparkException: Job 64 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1890)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1825)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2077)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2060)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2049)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:740)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2081)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2178)
	at org.apache.spark.rdd.RDD$$anonfun$fold$1.apply(RDD.scala:1098)
	a

In [ ]:

# iterate over vector to sum each "column"    
vec_sums = vec_df.rdd.fold([0]*num_cols, lambda a,b: [x + y for x, y in zip(a, b)])
vec_sums

# TODO: linear combination of positive - negative and compare the result to the "to recommend" tweet, transform result vector anyhow into a percentage

In [ ]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
#https://stackoverflow.com/questions/46758768/calculating-the-cosine-similarity-between-all-the-rows-of-a-dataframe-in-pyspark

mat = IndexedRowMatrix(
    data.select("engaging_user_id_idx", "normed_features")\
        .rdd.map(lambda row: IndexedRow(row.engaging_user_id_idx, row.normed_features.toArray()))).toBlockMatrix()
dot = mat.multiply(mat.transpose())
dot.toLocalMatrix().toArray()

In [4]:
sc.stop()

NameError: name 'sc' is not defined

Not fucking feasable to iterate naively  over 36000 features


from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml.linalg import SparseVector, DenseVector

# negative feedback: check if it exists
vec_df = positive_feedback.select('normed_features')
n = vec_df.first()[0].size

def sparse_to_array(v):
    v=DenseVector(v)
    ret = list([float(x)for x in v])
    return ret

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))

vec_df = vec_df.withColumn('normed_features_arr', sparse_to_array_udf('normed_features'))

d_plus = vec_df.agg(F.array(*[F.sum(F.col('normed_features_arr')[i]) for i in range(n)]).alias("sum"))
